In [ ]:
# Custom modules
from aoi import Project, AOI, Patcher
from eolearn_datafill import S2L2AImages, CustomInput, AddMask, DerivateProduct, MaskValidation, EstimatorParser
# Scikit-learn + LightGBM
import lightgbm as lgb
from sklearn import metrics
# eo-learn + sentinelhub
from eolearn.core import EOTask, EOPatch, EOExecutor, LinearWorkflow, FeatureType, OverwritePermission, SaveTask, MergeFeatureTask
from eolearn.features import LinearInterpolation, SimpleFilterTask, PredictPatch
from eolearn.geometry import VectorToRaster, PointSamplingTask, ErosionTask
from eolearn.io import ExportToTiff
from sentinelhub import CRS

In [ ]:
project = Project('jihozapad_brna')

# Choose study area: must be a geojson or a shapefile of
#a single feautre, which is a polygon of AOI.
aoi = AOI('basedata/jmk_aoi.geojson', crs=CRS.UTM_33N)
aoi.convert_desired_CRS()

# Splitting the area of interest.
patcher = Patcher(project)
patcher.get_xy_splitters(aoi.dimensions, patch_factor=1)
patcher.split_bboxes(aoi.shape)

# Create a GeoDataFrame of patches made according to the splitters and central patch selection.
# Initial patch can be selected by subset_patch.
patcher.get_patch_gdf(subset_patch=54, save=False)
patcher.get_patch_map(aoi.gdf)

In [ ]:
# Pre-processing cadastre reference map
# Loading this file takes around 3 minutes
LC = gpd.read_file('cadastre_south_moravian_region.shp')
# Copying dataset for modifications
land_cover = LC.copy()
# Reclassifying the information classes
reclassify = {
    2: 1,
    3: 0,
    4: 2,
    5: 3,
    6: 4,
    7: 5,
    10: 6,
    11: 7,
    13: 8,
    14: 0,
}

land_cover['druh_pozem'] = land_cover['druh_pozem'].map(reclassify) # Note: druh_pozem = lulc type on the estate
# Masking invalid geometries from the cadastre dataset
mask = (land_cover['geometry']!='shapely.geometry.polygon.Polygon')
land_cover = land_cover[mask]
# Adding roads to built-up areas
land_cover.loc[land_cover.zpusob_vyu.isin([15,16,18]), 'druh_pozem'] = 8
# Removing class 0: no-data/other surfaces
land_cover = land_cover[land_cover['druh_pozem']!=0]

In [ ]:
# Obtaining available Sentinel-2 L2A images and retrieving their metadata
images = S2L2AImages(patcher.gdf, project.FOLDER)
images.get_patches_bbox()

images.get_available(
    esa_sci_hub_credentials = ('username', 'password'), # Credentials to Copernicus Open Access Hub must be supplied
    date_range=('20190301', '20191130'), # Download range
    cloudcov=(0, 60) # Cloud coverage restriction
) 

# The possibility to select concrete images from the range
images.select() # No arguments means download all available
# Download images
images.download()

In [ ]:
# Desining the workflow to amend and process Sentinel-2, cadastre data
# and to almost prepare them for the classification
# Custom EOTask for adding and parsing downloaded images.
customS2L2A = CustomInput(
    images_gdf=images.downloaded, # Downloaded images dataframe
    path_to_images=project.FOLDER, # Path to downloaded images
    custom_input_name='BANDS' # Key name of input data in EOPatches
) 

# Custom EOTask for adding a mask from Sentinel-2 L2A 20m Scene Classification Layer (SCL)
# that is clipped and resampled to 10 m
addmask = AddMask(
    images_gdf=images.downloaded,
    path_to_images=project.FOLDER,
    mask_name='SCL_MASK'
)

# Original eo-learn EOTask for rasterizing cadastre LULC map
# and adding it to FeatureType.mask_timeless['LULC']
lulc_rasterization = VectorToRaster(
    land_cover, # Land cover GeoDataFrame
    (FeatureType.MASK_TIMELESS, 'LULC'), # FeatureType and name of the feature to save to EOPatches
    values_column='druh_pozem', # GeoDataFrame column to be used as a raster value
    raster_shape=(FeatureType.MASK, 'SCL_MASK'), # Make land cover to have same dimensions and cell size as SCL MASK
    raster_dtype=np.dtype(np.uint8) # NumPy array data type = unsigned 8 bit integer (meaningful for cadastre data)
)

# Original eo-learn EOTask for merging data along the band dimension
merging = MergeFeatureTask(
    {FeatureType.DATA: ['BANDS', 'NDVI']}, # Features to merge
    (FeatureType.DATA, 'FEATURES') # FeatureType and name of the feature to save to EOPatches
)

# Validation, according to the SCL mask
valid_data = MaskValidation(0.1) # Maximum threshold (10 %) of False pixels
# Original eo-learn for filtering images according to the MaskValidation
filtering = SimpleFilterTask(
    (FeatureType.MASK, 'SCL_MASK'), # FeatureType and name of the feature to save to EOPatches
    valid_data # CLMValidation results
)

# Original eo-learn for merging data along the band dimension
interpolation = LinearInterpolation(
    'FEATURES', # FeatureType data to be interpolated
    copy_features=[
        (FeatureType.MASK_TIMELESS, 'LULC'),
        (FeatureType.META_INFO)
    ], # Preserving some features in EOPatches
    mask_feature=(FeatureType.MASK, 'SCL_MASK'), # Masking data with the respective CLMs
    resample_range=(
        '2019-03-30', # First date of arbitrary range
        '2019-10-18', # Last date of arbitrary range
        10 # Interpolation step in days
    )
)

# Sampling pixels from patches for the estimator
spatial_sampling = PointSamplingTask(
    n_samples=50000, # Number of pixels to sample from each band in each time frame in each EOPatch
    ref_mask_feature='LULC', # Reference map (e.g. cadastre reference map)
    ref_labels=list(range(1,9)), # Unique information class labels from the reference map
    sample_features=[ # Specify which fields to sample
        (FeatureType.DATA, 'FEATURES'),
        (FeatureType.MASK_TIMELESS, 'LULC')
    ]
)

# Custom EOTask for removing no-date (Nans) from sampled pixels
nrm = NanRemover(
    sampled_feature_name='FEATURES',
    sampled_lulc_name='LULC_SAMPLED'
)

# Original eo-learn EOTask for saving EOPatches as npy files to disk
save = SaveTask(project.FOLDER, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

In [ ]:
 # Instantiating EOWorkflow
workflow = LinearWorkflow(
    customS2L2A,
    addmask,
    ndvi,
    ndwi,
    ndbi,
    lulc_rasterization,
    merging,
    filtering,
    interpolation,
    lulc_erosion,
    spatial_sampling,
    nrm,
    save
)

In [ ]:
# External parameters of the workflow
execution_args = patcher.gdf.apply(
    lambda row: {
        customS2L2A: {'patch_bbox': row[3]}, # row[3] = Patcher DataFrame position of bounding box column
        addmask: {'patch_bbox': row[3]},
        save: {'eopatch_folder': f'eopatch{row[2]}'} # row[2] = Patcher DataFrame position of bounding box ID column
    }, 
    axis=1).to_list()

# Instantiating EOExecutor
executor = EOExecutor(
    workflow, # The workflow defined above
    execution_args, # External execution arguments to feed to EOTasks
    save_logs=True, # Save detailed logs about what is happening
    logs_folder=project.FOLDER
)

# Running the workflow
executor.run(workers=9) # Specify multiprocessing division to CPU threads
executor.make_report() # Make an HTML report

In [ ]:
 # Loading EOPatches and feeding them to EstimatorParser class to get
# the feature vectors and information class labels
eopatches = np.array([EOPatch.load(os.path.join(project.FOLDER, f'eopatch{i}'), lazy_loading=True) for i in range(9)])

# Reducing feature space of training data
parse_training_data = EstimatorParser(
    eopatches, # Feed EOPatches
    patch_ids=[7,3,5,8,0], # Which EOPatches
    features='FEATURES_SAMPLED', # Which features to reduce
    classes='LULC_SAMPLED' # LULC to reduce
) 

# Reducing feature space of testing data
parse_testing_data = EstimatorParser(
    eopatches,
    patch_ids=[6,1,2,4],
    features='FEATURES_SAMPLED',
    classes='LULC_SAMPLED'
)

train_features, train_classes = parse_training_data()
test_features, test_classes = parse_testing_data()

In [ ]:
 # Adapted from Lubej (2019a, 2019b), eo-learn (2018)
#Set up training classes
labels_unique = np.unique(train_classes)

# LightGBM model with default parametres
model = lgb.LGBMClassifier(
    objective='multiclassova',
    num_class=len(labels_unique),
    metric='multi_logloss',
    random_state = 10
)

# Train the model
model.fit(train_features, train_classes)

# Testing the trained model on test features
prediction_test_set = model.predict(features_test)

# Obtaining pixel confusion matrix from predicted test samples and
# corresponding (but ground truth) test classes
confusion_matrix = metrics.confusion_matrix(prediction_test_set, test_classes)

In [ ]:
# Showcase of predicting an EOPatch
predict = PredictPatch(model, feature_name='PREDICTED_LULC')
eopatch_predicted = PredictPatch.execute(eopatch)

# Original EOTask for exporting predicted LULC to GEoTiff
export_tiff = ExportToTiff(
    (FeatureType.MASK_TIMELESS, 'PREDICTED_LULC'), 
    filename='predicted_eopatch1.tiff'
)

export_tiff.execute(eopatch_predicted)